In [3]:
import json
import csv
import pandas as pd
import numpy as np
import datetime
from bs4 import BeautifulSoup
import re
import sqlite3

from util import JsonLoader,dbsrh,dbexe
from conf import DefaultConfig

In [5]:
#配置
cfg_c=DefaultConfig()
cfg=cfg_c.cfg
n_chara=cfg['n_chara']
date_i=cfg["date_i"]
mysql=cfg["mysql"]["config"]
halo=cfg["halo"]

col=tuple(cfg["col"])
col_c=tuple(cfg["col_c"])

In [6]:
#连接数据库    
db=sqlite3.connect(database=mysql["database"]+".db")
cursor=db.cursor()
table_name="records"
sql_string = f"""CREATE TABLE {table_name} ({cols_string});"""

'''
sql="load data local infile \"E:/cal/data/rlt.csv\" into table record character set utf8  fields terminated by ','  lines terminated by '\r\n'   ignore 1 lines;"
dbexe(db,cursor,sql)
sql="delete from record where chara=\"野\";"
dbexe(db,cursor,sql)
'''

'\nsql="load data local infile "E:/cal/data/rlt.csv" into table record character set utf8  fields terminated by \',\'  lines terminated by \'\r\n\'   ignore 1 lines;"\ndbexe(db,cursor,sql)\nsql="delete from record where chara="野";"\ndbexe(db,cursor,sql)\n'

In [ ]:


json2csv("data/m.json","data/rlt.csv",d_halo=halo,col=col,col_c=col_c)
json2csv("data/i.json","data/rlt.csv",d_halo=halo,col=col,col_c=col_c,mode="a+")

In [169]:


#连接数据库    
db=pymysql.connect(host=mysql["host"],user=mysql["user"],password=mysql["password"],database=mysql["database"],local_infile=True)
cursor=db.cursor()

sql="load data local infile \"E:/cal/data/rlt.csv\" into table record character set utf8  fields terminated by ','  lines terminated by '\r\n'   ignore 1 lines;"
dbexe(db,cursor,sql)
sql="delete from record where chara=\"野\";"
dbexe(db,cursor,sql)



DB

In [ ]:
df=pd.read_csv("data/SSS.csv",encoding="utf-8")
#pd.set_option('display.max_columns',None)
name=[c for c in df.columns]
date_t=datetime.date(date_i[0],date_i[1],date_i[2])

sql="SELECT * FROM cnt;"
rlt= dbsrh(db,cursor,sql)


if rlt is not None:
    weight=dict()
    for r in rlt:
        weight[r[0]]=r[1]

# 关闭不使用的游标对象
cursor.close()
# 关闭数据库连接
db.close()

输出加权胜率

In [ ]:
weight=[float(weight[(name[2*i*(n_chara+1)].strip("\t"))[0]])/sum(weight.values()) for i in range(n_chara)]
weight=np.array(weight)

dp=len(df.index)
avg_i=dict()
avg_w=dict()
for k in range(dp):
    c_date=date_t+datetime.timedelta(k)
    c_date="{0}-{1}".format(c_date.month,c_date.day)
    avg_i[c_date]=dict()
    avg_w[c_date]=dict()
    for i in range(n_chara*2):
        di=[]
        for j in range(n_chara*i+1+i,n_chara*(i+1)+1+i):
            di.append(float(df[name[j]][k].strip('%'))/100)
        arr=np.array(di)
        avg_i[c_date][name[i*(n_chara+1)]]=np.mean(arr)
        rlt=arr*weight
        #df[name[i*(n_chara+1)]][k]=round(rlt.sum(),3)
        avg_w[c_date][name[i*(n_chara+1)]]=round(rlt.sum(),3)
#df.to_csv("out.csv",columns=[name[i*(n_chara+1)] for i in range(n_chara*2) ],encoding="gbk")
avg_w=pd.DataFrame(avg_w)
avg_w.sort_values(by=c_date,ascending=False,inplace=True)
avg_w.to_csv("data/out_w.csv",encoding="gbk")
avg_i=pd.DataFrame(avg_i)
avg_i.sort_values(by=c_date,ascending=False,inplace=True)
avg_i.to_csv("data/out_i.csv",encoding="gbk")

In [39]:

with open("data/m.json",'r',encoding="utf-8") as f:
    datas=json.load(f)
    fcsv=[("id","enemyname","char","charlevel","time")]
    for data in datas["data"]["data"][0]["rows"]:
        fcsv.append((data["id"],data["enemyname"],data["char"],data["charlevel"],data["time"]))
    with open("data/rlt.csv","w",newline="",encoding="utf-8") as f2:
        writer=csv.writer(f2)
        writer.writerows(fcsv)
    data=datas["data"]["data"][0]["rows"][0]

def json2csv(input,out,col=(),mode="w",encoding="utf-8"):
    with open(input,'r',encoding=encoding) as f:
        datas=json.load(f)
        if mode=="w":
            fcsv=[col]
        else:
            fcsv=[]
        for data in datas["data"]["data"][0]["rows"]:
            fcsv.append(tuple([data[c] for c in col]))
        with open(out,mode=mode,newline="",encoding=encoding) as f2:
            writer=csv.writer(f2)
            writer.writerows(fcsv)

col=("id","enemyname","char","charlevel","time")

json2csv("data/m.json","data/rlt.csv",col=col)
json2csv("data/i.json","data/rlt.csv",col=col,mode="a+")


print(data["id"],data["enemyname"],data["char"],data["charlevel"],data["time"])
print()

007a508e602eca098b9887497a5ec03b wwoonn 艾 826 1671465600000



In [49]:
import yaml
import os

class DefaultConfig():
    def __init__(self,path,encoding="utf-8"):
        with open(path,"r",encoding=encoding) as f:
            self.cfg=yaml.safe_load(f)
    def loaddata(self,fname) :
        try:
            cfg_f=self.cfg["path_data"]
            path_w=cfg_f["path_data"]
            return os.path.join(path_w,cfg_f[fname])
        except ValueError as e:
            print(e)
            return

path="conf/config.yaml"
cfg=DefaultConfig(path=path)
print(cfg.loaddata("json_i"))

data/i.json


In [163]:
with open("data/m.json",'r',encoding="utf-8") as f:
    datas=json.load(f)
    fcsv=[("id","enemyname","char","charlevel","time","log")]
    for data in datas["data"]["data"][0]["rows"]:
        fcsv.append((data["id"],data["enemyname"],data["char"],data["charlevel"],data["time"],data["log"]))
    test=(fcsv[9][5])
bs = BeautifulSoup(test,"html.parser")
eq=bs.find_all(name="button",attrs={"data-original-title":True},limit=8)[4:]
for e in eq:
    print(e.attrs['data-original-title'],e.text)
tf=bs.find(name="div",attrs={"class":"col-md-7 fyg_tl"})
w_match=re.findall("\d+",tf.text)[1:]
print(w_match)
halo=bs.find(name="div",attrs={"class":"col-md-5 fyg_tr"})
w_match=re.findall(r"(?<=\|)[^|]\S{3}(?=|)",halo.text)
print(halo)
print(w_match)

狂信者的荣誉之刃 296
噬魔戒指 288
复苏战衣 264
探险者耳环 295
['850', '762211', '10290', '6801', '19899', '12', '1758', '1758']
<div class="col-md-5 fyg_tr">
<button class="btn fyg_colpzbg fyg_mp3" data-original-title="狂信者的荣誉之刃" data-placement="bottom" data-toggle="tooltip" style="width:47px;height:47px;background-image:url(https://p.inari.site/guguicons/test/eq/%E7%8B%82%E4%BF%A1%E8%80%85%E7%9A%84%E8%8D%A3%E8%AA%89%E4%B9%8B%E5%88%83/4.gif);" title="" type="button"><br/>296</button><button class="btn fyg_colpzbg fyg_mp3" data-original-title="噬魔戒指" data-placement="bottom" data-toggle="tooltip" style="width:47px;height:47px;background-image:url(https://p.inari.site/guguicons/test/eq/%E5%99%AC%E9%AD%94%E6%88%92%E6%8C%87/4.gif);" title="" type="button"><br/>288</button><button class="btn fyg_colpzbg fyg_mp3" data-original-title="复苏战衣" data-placement="bottom" data-toggle="tooltip" style="width:47px;height:47px;background-image:url(https://p.inari.site/guguicons/test/eq/%E5%A4%8D%E8%8B%8F%E6%88%98%E8%A1%A3/4.gif

In [139]:
print(bs.prettify())

<div class="row">
 <div class="row">
  <div class="col-md-6">
   <div class="alert alert-danger" style="background-color:#ffffff;border:1px #EA644A solid;">
    <div class="row">
     <div class="col-md-5 fyg_tl">
      <button class="btn fyg_colpzbg fyg_mp3" data-original-title="反叛者的刺杀弓" data-placement="bottom" data-toggle="tooltip" style="width:47px;height:47px;background-image:url(https://p.inari.site/guguicons/test/eq/%E5%8F%8D%E5%8F%9B%E8%80%85%E7%9A%84%E5%88%BA%E6%9D%80%E5%BC%93/4.gif);" title="" type="button">
       <br/>
       291
      </button>
      <button class="btn fyg_colpzbg fyg_mp3" data-original-title="海星戒指" data-placement="bottom" data-toggle="tooltip" style="width:47px;height:47px;background-image:url(https://p.inari.site/guguicons/test/eq/%E6%B5%B7%E6%98%9F%E6%88%92%E6%8C%87/4.gif);" title="" type="button">
       <br/>
       297
      </button>
      <button class="btn fyg_colpzbg fyg_mp3" data-original-title="复苏战衣" data-placement="bottom" data-toggle="tooltip"

In [144]:
str=r"""
<div class="col-md-7 fyg_tl">
      <span class="fyg_f18">
       ster1937（希 Lv.850）
      </span>
      <br/>
      [生命:762211] [护盾:10290]
      <br/>
      [速度:6801]
      <br/>
      [物攻:19899] [魔攻:12]
      <br/>
      [物防:1758] [魔防:1758]
      <br/>
     </div>
"""

w_match=re.findall("(?<=:)\d+(?=])",tf.text)
print(w_match)

line = "Cats are smarter than dogs"
 
matchObj = re.match( r'.*', line, re.M|re.I)
 
if matchObj:
   print ("matchObj.group() : ",matchObj.group())


['762211', '10290', '6801', '19899', '12', '1758', '1758']
matchObj.group() :  Cats are smarter than dogs


In [134]:
def json2csv(input,out,col=(),col_c=(),mode="w",encoding="utf-8"):
        with open(input,'r',encoding=encoding) as f:
            datas=json.load(f)
            if mode=="w":
                fcsv=[col]+[col_c]
                print(fcsv)
cfg_c=DefaultConfig()
cfg=cfg_c.cfg
n_chara=cfg['n_chara']
date_i=cfg["date_i"]
mysql=cfg["mysql"]["config"]

col=tuple(cfg["col"])
col_c=tuple(cfg["col_c"])

json2csv("data/m.json","data/rlt.csv",col=col,col_c=col_c)
json2csv("data/i.json","data/rlt.csv",col=col,col_c=col_c,mode="a+")

[('id', 'enemyname', 'char', 'charlevel', 'time'), ('weapon', 'lv_w', 'bracelet', 'lv_b', 'armor', 'lv_a', 'helmet', 'lv_h', 'halo')]


In [2]:
import requests
s = requests.session()
s.keep_alive=False
url="https://www.momozhen.com/fyg_stat.php"
headers = {'user-agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Mobile Safari/537.36 Edg/109.0.1518.61'}
r=requests.get(url,verify=False,headers=headers)

ProxyError: HTTPSConnectionPool(host='www.momozhen.com', port=443): Max retries exceeded with url: /fyg_stat.php (Caused by ProxyError('Cannot connect to proxy.', FileNotFoundError(2, 'No such file or directory')))

In [31]:
import json
import csv
import re
from bs4 import BeautifulSoup

class JsonLoader():
    def __init__(self,inputs,d_halo,col=[],col_c=[],encoding="utf-8") -> None:
        self.encoding=encoding
        self.d_halo=d_halo
        self.col=col
        self.col_c=col_c
        self.encoding=encoding
        self.loader=[self.json_proc(inputs[0],d_halo,col,col_c,mode="w",encoding=encoding)]
        for i in range(1,len(inputs)):
            self.loader.append(self.json_proc(inputs[i],d_halo,col,col_c,mode="a+",encoding=encoding))

    def json_add(self,input):
        self.mode="a+"
        ld=self.json_proc(input,d_halo=self.d_halo,col=self.col,col_c=self.col_c,mode="a+",encoding=self.encoding)
        self.loader.append(ld)
    
    def json2csv(self,out):
        with open(out,mode="w",newline="",encoding=self.encoding) as f2:
            writer=csv.writer(f2)
            for ld in self.loader:
                writer.writerows(ld)
    def createdb(self):
        db=sqlite3.connect(database=mysql["database"]+".db")
        cursor=db.cursor()
        table_name="records"
        sql_string = f"""CREATE TABLE {table_name} ({",".join(self.loader[0][0])});"""
        cursor.execute(sql_string)
        db.commit()
        db.close()

    def json2db(self):
        db=sqlite3.connect(database=mysql["database"]+".db")
        cursor=db.cursor()
        for ld in self.loader:
            table_name="records"
            cols=self.loader[0][0]
            format_v=','.join(['?'] * len(cols))
            sql_string = f"""INSERT INTO {table_name} ({",".join(cols)}) VALUES ({format_v});""" 
            cursor.executemany(sql_string, )
        db.commit()
        db.close()

    def json_proc(self,input,d_halo,col=[],col_c=[],mode="w",encoding="utf-8"):
        self.mode=mode
        self.encoding=encoding
        with open(input,'r',encoding=encoding) as f:
            datas=json.load(f)
            if mode=="w":
                loader=[tuple(col+col_c)]
            else:
                loader=[]
            for data in datas["data"]["data"][0]["rows"]:
                if data["char"]=="野怪":
                    pass
                else:
                    bs = BeautifulSoup(data["log"],"html.parser")
                    l_eq=bs.find_all(name="button",attrs={"data-original-title":True},limit=8)[4:]
                    eq=list()
                    for e in l_eq:
                        eq+=[e.attrs['data-original-title'],e.text ]    #装备
                    tf=bs.find(name="div",attrs={"class":"col-md-7 fyg_tl"})    #属性
                    w_match=re.findall("(?<=:)\d+(?=])",tf.text)
                    halo=bs.find(name="div",attrs={"class":"col-md-5 fyg_tr"})  #光环
                    s_halo=re.findall(r"(?<=\|)[^|]\S{3}(?=|)",halo.text)
                    bit_h=0
                    for h in s_halo:
                        bit_h+=(2**d_halo[h])
                    loader.append(tuple([data[c] for c in col]+eq+w_match+[str(s_halo)]+[bit_h]))
            return loader

if __name__=="__main__":
    from conf import DefaultConfig
    #配置
    cfg_c=DefaultConfig()
    cfg=cfg_c.cfg
    n_chara=cfg['n_chara']
    date_i=cfg["date_i"]
    mysql=cfg["mysql"]["config"]
    halo=cfg["halo"]

    col=tuple(cfg["col"])
    col_c=tuple(cfg["col_c"])
    jloader=JsonLoader(["data/f.json",],d_halo=halo,col=col,col_c=col_c)
    jloader.json2db()



AttributeError: 'JsonLoader' object has no attribute 'cursor'

In [14]:
jloader.json2db()

[('id', 'enemyname', 'char', 'charlevel', 'time', 'weapon', 'lv_w', 'bracelet', 'lv_b', 'armor', 'lv_a', 'helmet', 'lv_h', 'hp', 'mp', 'spd', 'patk', 'matk', 'pdef', 'mdef', 'halo', 'bit_h'), ('007a508e602eca098b9887497a5ec03b', 'wwoonn', '艾', '826', 1671465600000, '幽梦匕首', '287', '秃鹫手环', '296', '战线支撑者的荆棘重甲', '263', '探险者耳环', '285', '625016', '92', '1542', '13055', '189', '3965', '4233', "['启程之誓', '圣盾祝福', '铁甲尖刺', '点到为止', '荧光护盾', '钝化锋芒']", 84183042), ('007e26d8f73f5c28c2fe5bf5346aa327', '血萌', '琳', '850', 1671811200000, '荆棘盾剑', '270', '噬魔戒指', '294', '战线支撑者的荆棘重甲', '291', '萌爪耳钉', '280', '1272136', '3842', '3', '8270', '12', '5227', '3897', "['等级压制', '圣盾祝福', '伤口恶化', '铁甲尖刺', '荧光护盾', '钝化锋芒']", 83929120), ('00965714471e5b8c3f6200f5b141e2e5', 'frankmelody', '希', '850', 1671811200000, '反叛者的刺杀弓', '298', '噬魔戒指', '298', '战线支撑者的荆棘重甲', '300', '探险者耳环', '288', '1234143', '90', '267', '28214', '36', '3888', '2287', "['破壁之心', '伤口恶化', '铁甲尖刺', '忍无可忍', '纸薄命硬', '钝化锋芒']", 68264000), ('00e43efc676e86f316ef2b7390